In [ ]:
import time, re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# === Setup Driver ===
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-notifications")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# === Login Manual ===
driver.get('https://www.instagram.com/')
print("🔐 Silakan login secara manual. Tunggu 30 detik...")
time.sleep(30)

# === Profil Target ===
username_target = 'batikkultur'
profile_url = f'https://www.instagram.com/{username_target}/'
driver.get(profile_url)
print(f"\n🔍 Mengakses profil: {profile_url}")
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h2")))
time.sleep(3)

# === Ambil data profil ===
print("\n📌 Mengambil data profil...")
username = username_target
display_name, kategori_profil, bio, tautan = 'N/A', 'N/A', 'N/A', 'N/A'
total_posts, total_followers, total_following = 'N/A', 'N/A', 'N/A'

try:
    display_name = driver.find_element(By.CSS_SELECTOR, "section.xc3tme8 > div:nth-child(1) span").text.strip()
except: print("⚠️ Display name tidak ditemukan.")

try:
    kategori_profil = driver.find_element(By.CSS_SELECTOR, "div._ap3a._aaco._aacu._aacy._aad6._aade").text
except: print("⚠️ Kategori profil tidak ditemukan.")

try:
    tautan = driver.find_element(By.CSS_SELECTOR, "a[href*='l.instagram.com']").text
except: print("⚠️ Tautan tidak ditemukan.")

try:
    for el in driver.find_elements(By.CSS_SELECTOR, "span._ap3a"):
        txt = el.text.strip()
        if '@' in txt or 'WA' in txt or len(txt) > 20:
            bio = txt.replace('\n', ' ')
            break
except: print("⚠️ Bio tidak ditemukan.")

try:
    stats = driver.find_elements(By.CSS_SELECTOR, "ul.x78zum5 > li.xl565be")
    for stat in stats:
        t = stat.text
        if 'posts' in t: total_posts = t.split(' ')[0]
        elif 'followers' in t:
            try: total_followers = stat.find_element(By.CSS_SELECTOR, "span[title]").get_attribute('title')
            except: total_followers = t.split(' ')[0]
        elif 'following' in t: total_following = t.split(' ')[0]
except: print("⚠️ Gagal mengambil statistik.")

print(f"✅ Username        : {username}")
print(f"✅ Display Name    : {display_name}")
print(f"✅ Kategori        : {kategori_profil}")
print(f"✅ Bio             : {bio}")
print(f"✅ Tautan          : {tautan}")
print(f"📊 Statistik       : {total_posts} posts, {total_followers} followers, {total_following} following")

# === Klik Tab Reels ===
print("\n➡️ Menuju tab Reels...")
driver.get(f"https://www.instagram.com/{username_target}/reels/")
time.sleep(5)

# === Scroll dan Ambil Link Reels ===
scroll_times = 6
reels_summary_data, scraped_links = [], set()
print(f"\n🔁 Mulai scroll ({scroll_times}x)...")

for i in range(scroll_times):
    reel_rows = driver.find_elements(By.CSS_SELECTOR, "div._ac7v")
    print(f"🔂 Scroll ke-{i+1}: {len(reel_rows)} baris ditemukan.")

    for row in reel_rows:
        for link_element in row.find_elements(By.TAG_NAME, "a"):
            link = link_element.get_attribute("href")
            if not link or link in scraped_links or "/reel/" not in link: continue

            try:
                view_element = link_element.find_element(By.CSS_SELECTOR, "div._aajy span > span")
                views = view_element.text
            except NoSuchElementException:
                views = ''

            print(f"🎞️ {link} | Views: {views} ")
            reels_summary_data.append({'url_reel': link, 'views': views})
            scraped_links.add(link)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)

print(f"\n✅ Total reels ditemukan: {len(reels_summary_data)}")

# === Tahap Kunjungan Masing-masing Reel ===
print("\n🔍 Mengambil detail dari masing-masing Reel...")
all_data = []

for idx, reel in enumerate(reels_summary_data):
    link = reel['url_reel']
    views = reel['views']
    id_reel = link.split('/')[-2]

    print(f"\n➡️ {idx+1}/{len(reels_summary_data)} - {link}")
    driver.get(link)
    try:
        # Tunggu hingga elemen H1 (caption) ada
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
        time.sleep(2) # Beri waktu tambahan agar meta tag termuat
    except TimeoutException:
        print("⚠️ Halaman tidak termuat, skip.")
        continue

    caption, hashtags, upload_time = '', '', ''
    likes, comments_count = '0', '0'
    comments_list = []

    # **MODIFIKASI: Ambil Likes dan Comments Count dari Meta Tag**
    try:
        meta_tag_content = driver.find_element(By.CSS_SELECTOR, "meta[property='og:description']").get_attribute('content')
        
        # Ekstrak Likes
        likes_match = re.search(r'([\d,.]+)\s+likes', meta_tag_content)
        if likes_match:
            likes = likes_match.group(1).replace(',', '').replace('.', '')
        
        # Ekstrak Comments Count
        comments_match = re.search(r'([\d,.]+)\s+comments', meta_tag_content)
        if comments_match:
            comments_count = comments_match.group(1).replace(',', '').replace('.', '')
        
        print(f"👍 Meta Likes: {likes} | 💬 Meta Comments: {comments_count}")

    except NoSuchElementException:
        print("⚠️ Meta tag 'og:description' tidak ditemukan.")
    except Exception as e:
        print(f"⚠️ Error saat mengambil data dari meta tag: {e}")


    # Caption + Hashtag
    try:
        caption_full = driver.find_element(By.TAG_NAME, 'h1').text.strip()
        hashtags_found = re.findall(r"#\w+", caption_full)
        hashtags = ' '.join(hashtags_found)
        caption = re.sub(r'#\w+\s*', '', caption_full).strip()
    except:
        print("⚠️ Caption tidak ditemukan.")

    # Upload time
    try:
        upload_time = driver.find_element(By.TAG_NAME, 'time').get_attribute('datetime')
    except: pass

    # Komentar (Logika pengambilan teks komentar tetap sama)
    print("💬 Mengambil teks komentar...")
    while True:
        try:
            xpath = "//button[.//*[contains(@aria-label, 'Load more comments') or contains(@aria-label, 'Muat komentar lainnya')]]"
            load_more = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, xpath)))
            driver.execute_script("arguments[0].click();", load_more)
            time.sleep(3) # Kurangi waktu tunggu sedikit
        except: break

    try:
        comment_elements = driver.find_elements(By.CSS_SELECTOR, "ul._a9ym span._aade")
        comments_list = [el.text.strip() for el in comment_elements if el.text.strip()]
    except: pass
    
    final_comments = comments_list[:50]
    print(f"✅ Berhasil mengambil {len(final_comments)} teks komentar.")

    # === Simpan ke list ===
    data_profil = [username, display_name, kategori_profil, bio, tautan, total_posts, total_followers, total_following]
    if not final_comments:
        all_data.append(data_profil + [id_reel, link, views, likes, comments_count, caption, hashtags, upload_time, ''])
    else:
        for comment in final_comments:
            all_data.append(data_profil + [id_reel, link, views, likes, comments_count, caption, hashtags, upload_time, comment])

# === Simpan ke Excel ===
# **MODIFIKASI: Mengganti nama kolom menjadi 'comments_count'**
df = pd.DataFrame(all_data, columns=[
    'username', 'display_name', 'kategori_profil', 'bio', 'tautan',
    'total_posts', 'total_followers', 'total_following',
    'id_reel', 'url_reel', 'views', 'likes', 'comments_count',
    'caption', 'hashtags', 'upload_time', 'comment'
])

output_file = f'hasil_scrape_reels_lengkap_{username}.xlsx'
df.to_excel(output_file, index=False)
print(f"\n📁 File berhasil disimpan: {output_file}")

driver.quit()